### Import modules 

In [1]:
import os 
import shutil 
import logging 
import time
from datetime import datetime

### Create functions 

In [2]:
def setup_logging(log_file):
    '''Create a logger and set it´s parameters'''
    
    # Create logger
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG) # set the scale of log messages. 

    # Create console handler 
    log_records_handler = logging.StreamHandler() # creates a handler that sends log records to a stream 
    log_records_handler.setLevel(logging.DEBUG)

    # Create file handler 
    file_handler = logging.FileHandler(log_file, encoding='utf-8')  # Set encoding 'utf-8'
    file_handler.setLevel(logging.DEBUG)

    # Create and set formatter
    formatter = logging.Formatter('%(asctime)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S') # specify formates
    log_records_handler.setFormatter(formatter)
    file_handler.setFormatter(formatter)

    # Add handlers to logger
    logger.addHandler(log_records_handler)
    logger.addHandler(file_handler)

def synchronize_folders(source_folder, replica_folder):
    '''Function to synchronize files'''
    # if replica folder does not exist, create it
    if not os.path.exists(replica_folder):
        os.makedirs(replica_folder)

    # Iterate through all files and subdirectories in the source folder
    for item in os.listdir(source_folder):
        source_item = os.path.join(source_folder, item)
        replica_item = os.path.join(replica_folder, item)

        # If it's a file, copy it to the replica 
        if os.path.isfile(source_item):
            if not os.path.exists(replica_item) or \
                    os.stat(source_item).st_mtime - os.stat(replica_item).st_mtime > 1: # if the modification time > than 1 second
                shutil.copy2(source_item, replica_item)
                logging.info(f"File copied: {source_item} -> {replica_item}")
                
        # If it's a directory, synchronize it via inner calling of the same function
        elif os.path.isdir(source_item):
            # inner calling of the same function 
            synchronize_folders(source_item, replica_item)

    # Delete any files or directories in the replica folder that don't exist in the source folder
    for item in os.listdir(replica_folder):
        replica_item = os.path.join(replica_folder, item)
        source_item = os.path.join(source_folder, item)

        # If the item exists in replica but not in source, delete the item
        if not os.path.exists(source_item):
            if os.path.isfile(replica_item):
                os.remove(replica_item)
                logging.info(f"File removed: {replica_item}")
            elif os.path.isdir(replica_item):
                shutil.rmtree(replica_item)
                logging.info(f"Directory removed: {replica_item}")

def main(source_folder, replica_folder, interval, log_file):
    '''Call all functions together
    Runs forever if not interupted by keyboard input.'''
    setup_logging(log_file)
    logging.info("Starting synchronization process...")

    try:
        while True:
            synchronize_folders(source_folder, replica_folder)
            logging.info("Folders synchronized successfully.")
            time.sleep(interval)
            
    # Keyboard Interupt - press Ctrl+C in the terminal         
    except KeyboardInterrupt:
        logging.info("Synchronization process stopped by user.")

### Insert interval and file paths   

In [2]:
# synchronization interval (in seconds) BY DEFAULT
interval = input('Insert synchronization interval in seconds:')  

# SET the folders
source_folder = input('Insert source folder path:')  
replica_folder = input('Insert replica folder path:')  

# Set the log file (like: "sync_logger.txt")
log_file = input('Insert log file path:')

Insert synchronization interval in seconds:3600
Insert source folder path:blab
Insert replica folder path:blal
Insert log file path:blal


### Run the program 

In [ ]:
if __name__ == "__main__":
    main(source_folder, replica_folder, interval, log_file)

2024-02-26 15:52:34 - Starting synchronization process...
2024-02-26 15:52:34 - File copied: C:/Users/Matěj/Desktop/My_source\matej_1\factorials.png -> C:/Users/Matěj/Desktop/My_replica\matej_1\factorials.png
2024-02-26 15:52:34 - File copied: C:/Users/Matěj/Desktop/My_source\matej_1\Matej_2\test_2.txt -> C:/Users/Matěj/Desktop/My_replica\matej_1\Matej_2\test_2.txt
2024-02-26 15:52:34 - File copied: C:/Users/Matěj/Desktop/My_source\náklady byt 2023.png -> C:/Users/Matěj/Desktop/My_replica\náklady byt 2023.png
2024-02-26 15:52:35 - File copied: C:/Users/Matěj/Desktop/My_source\test_1.txt -> C:/Users/Matěj/Desktop/My_replica\test_1.txt
2024-02-26 15:52:35 - Folders synchronized successfully.
